In [1]:
import pandas as pd
import numpy as np
from collections import Counter
data = pd.read_csv('data/data.csv')
#data = pd.DataFrame.from_csv('data/data.csv')

## Get Id's where Age is 0 

In [2]:
zeroAge = data[data['Age'] == 0].index.tolist()

In [3]:
runnerId = [data.iloc[x]['Id'] for x in zeroAge]

In [4]:
for n in runnerId:
    print data[data['Id'] == n]

      Id  Age  Rank  Year   Time  Pace  Sex
488  308   45  1204  2016  15137   578    0
489  308    0  1357  2014  15975   609    0
        Id  Age  Rank  Year   Time  Pace  Sex
2041  1229   48  2717  2009  22803   870    1
2042  1229   47  2414  2008  20344   776    1
2043  1229   53  3194  2014  21215   809    1
2044  1229   46  2054  2007  19896   759    1
2045  1229   51  3870  2012  23302   889    1
2046  1229    0  3332  2011  20990   801    1
2047  1229   50  2208  2006  21090   804    1
2048  1229   50  3482  2011  22896   873    1
2049  1229   49  2811  2010  23191   885    1
2050  1229   44  1566  2005  19757   754    1
2051  1229   55  2783  2016  20941   800    1
2052  1229   50  2748  2015  25435   970    1
         Id  Age  Rank  Year   Time  Pace  Sex
10743  6491   43   217  2006  12496   476    1
10744  6491   50   396  2015  13172   502    1
10745  6491    0   264  2012  12452   475    1
10746  6491   42    84  2005  11596   442    1
         Id  Age  Rank  Year   Time

The following runner id's represent instances where the age is unknown, but the id only appears once, therefore age cannot be corrected: [16673,16954,17902,18202,22942]. To deal with these, the unknown age of the id was set to the average age of all runners who ranked within a range of +/- 50 of the runner in that particular year.

In [5]:
def correctSingleInstanceAge(runnerId,data):
    rank = int(data[data['Id'] == runnerId]['Rank'])
    year = int(data[data['Id'] == runnerId]['Year'])
    rangeOfRunners = data[(data['Rank']>rank-50) & (data['Rank']<rank+50)& (data['Year'] == year)]
    avgAge = rangeOfRunners['Age'].sum()/rangeOfRunners['Age'].size

    index = data[data['Id']==runnerId].index
    data.set_value(index, 'Age', avgAge)
    return


In [6]:
ageCorrectionRunnerIds = [16673,16954,17902,18202,22942]
for n in ageCorrectionRunnerIds:
    correctSingleInstanceAge(n,data)

Runner id's [23714,26270] represent unknown identities, despite clearly being different people that ran in the same year. This represents 6 people in 2013 and 4 people in 2014.

In [7]:
zeroAge = data[(data['Age'] == 0)&(data['Id'] != 26270) &(data['Id'] != 23714) ].index.tolist()
runnerId = [data.iloc[x]['Id'] for x in zeroAge]
for i in runnerId:
    print data[data['Id'] == i].sort_values(by='Year')

      Id  Age  Rank  Year   Time  Pace  Sex
489  308    0  1357  2014  15975   609    0
488  308   45  1204  2016  15137   578    0
        Id  Age  Rank  Year   Time  Pace  Sex
2050  1229   44  1566  2005  19757   754    1
2047  1229   50  2208  2006  21090   804    1
2044  1229   46  2054  2007  19896   759    1
2042  1229   47  2414  2008  20344   776    1
2041  1229   48  2717  2009  22803   870    1
2049  1229   49  2811  2010  23191   885    1
2046  1229    0  3332  2011  20990   801    1
2048  1229   50  3482  2011  22896   873    1
2045  1229   51  3870  2012  23302   889    1
2043  1229   53  3194  2014  21215   809    1
2052  1229   50  2748  2015  25435   970    1
2051  1229   55  2783  2016  20941   800    1
         Id  Age  Rank  Year   Time  Pace  Sex
10746  6491   42    84  2005  11596   442    1
10743  6491   43   217  2006  12496   476    1
10745  6491    0   264  2012  12452   475    1
10744  6491   50   396  2015  13172   502    1
         Id  Age  Rank  Year   Time

For the remaining runner Ids, the participant ran in at least one other year where the age information is recorded. By taking the difference in the years, this offset is added to the recorded age.

In [8]:
def offsetAge(idRecorded,idError,data):
    ageRecorded = data.loc[idRecorded]['Age']
    yearRecorded = data.loc[idRecorded]['Year']
    yearError = data.loc[idError]['Year']
    offset = yearError - yearRecorded
    data.set_value(idError, 'Age', ageRecorded+offset)
    return

In [9]:
offsetAge(488,489,data)
offsetAge(2049,2046,data)
offsetAge(10743,10745,data)
offsetAge(13066,13065,data)
offsetAge(24232,24231,data)

In [10]:
data[(data['Id'] != 23714) & (data['Id'] != 26270) & (data['Age']==0)]

,Id,Age,Rank,Year,Time,Pace,Sex


## Smallest Age

In [11]:
data[(data['Id'] != 23714) & (data['Id'] != 26270)]['Age'].min()

9

In [12]:
data[data['Age']==9]

,Id,Age,Rank,Year,Time,Pace,Sex
31786,23689,9,2339,2013,6994,533,1


In [13]:
data[data['Id']==23689]

,Id,Age,Rank,Year,Time,Pace,Sex
31786,23689,9,2339,2013,6994,533,1
31787,23689,45,2339,2013,6994,533,1


Participants ran in the same year with the same id/time/pace/sex. Id 31786 can be perceived as an error and eliminated.

In [14]:
data = data.drop([31786])

In [15]:
data[(data['Id'] != 23714) & (data['Id'] != 26270)]['Age'].min()

10

# Minimum age of Miami Marathon is 11. Check outliers

For the rows with age == 10, these runners do not run in any other years. Simply remove these data points and consider them as outliers. 

In [16]:
ageoutliers = data[data['Age']==10].index.tolist()

In [17]:
ageRunnerID = []
for ageID in ageoutliers:
    ageRunnerID.append(data.loc[ageID]['Id'])

for agerunnerid in ageRunnerID:
    print data[data['Id']==agerunnerid]

for ageID in ageoutliers:
    data = data.drop(ageID)

         Id  Age  Rank  Year   Time  Pace  Sex
12096  7493   10   817  2006  14788   564    1
          Id  Age  Rank  Year   Time  Pace  Sex
24513  17305   10   500  2010  13841   528    1


## Age distribution

In [18]:
ages = data['Age']
ageCount = Counter(ages)
ageCount

Counter({0: 10,
         12: 3,
         13: 3,
         14: 4,
         15: 77,
         16: 61,
         17: 92,
         18: 91,
         19: 109,
         20: 409,
         21: 252,
         22: 324,
         23: 456,
         24: 608,
         25: 1373,
         26: 771,
         27: 779,
         28: 906,
         29: 1026,
         30: 1883,
         31: 1109,
         32: 1108,
         33: 1125,
         34: 1159,
         35: 2278,
         36: 1233,
         37: 1203,
         38: 1214,
         39: 1344,
         40: 2422,
         41: 1166,
         42: 1170,
         43: 1085,
         44: 1054,
         45: 1791,
         46: 945,
         47: 796,
         48: 775,
         49: 730,
         50: 1272,
         51: 558,
         52: 505,
         53: 462,
         54: 365,
         55: 593,
         56: 259,
         57: 263,
         58: 230,
         59: 204,
         60: 295,
         61: 141,
         62: 138,
         63: 106,
         64: 83,
         65: 95,
     

In [19]:
ageoutliers = data[data['Age']==15].index.tolist()
ageRunnerID = []
for ageID in ageoutliers:
    ageRunnerID.append(data.loc[ageID]['Id'])

for agerunnerid in ageRunnerID:
    if(data[data['Id']==agerunnerid]['Age'].size>1):
        print data[data['Id']==agerunnerid]

#for ageID in ageoutliers:
#    data = data.drop(ageID)

      Id  Age  Rank  Year   Time  Pace  Sex
457  287   20  2932  2016  22658   865    1
458  287   15  2011  2015  17820   680    1
        Id  Age  Rank  Year   Time  Pace  Sex
1759  1066   15  1409  2015  15926   607    1
1760  1066   15  2467  2016  19108   730    1
        Id  Age  Rank  Year   Time  Pace  Sex
1759  1066   15  1409  2015  15926   607    1
1760  1066   15  2467  2016  19108   730    1
        Id  Age  Rank  Year   Time  Pace  Sex
2831  1703   15  2324  2015  19430   741    1
2832  1703   40   505  2010  13855   528    1
2833  1703   35   987  2016  14534   555    1
2834  1703   18  2967  2014  20175   770    1
        Id  Age  Rank  Year   Time  Pace  Sex
3143  1918   40  2193  2015  18780   716    0
3144  1918   15  2517  2016  19324   738    0
          Id  Age  Rank  Year   Time  Pace  Sex
19494  13321   15  1925  2010  17646   673    1
19495  13321   16  2433  2011  17275   659    1
19496  13321   14  2555  2009  20692   789    1
          Id  Age  Rank  Year   

# *** NEED TO FIX AGES ?? ***

## Force Ages to increments of 5

In [20]:
for i in range (data['Age'].size):
    data.iloc[i]['Age'] = data.iloc[i]['Age'] - data.iloc[i]['Age']%5

In [21]:
data[(data['Id'] != 23714) & (data['Id'] != 26270)]['Age'].min()

10

## Add weather data
Temperature in deg. C at 6:53 or 6:59 AM

In [22]:
temp = {
    '2003':10.6,
    '2004':20.6,
    '2005':18.9,
    '2006':21.1,
    '2007':22.2,
    '2008':17.8,
    '2009':14.4,
    '2010':21.7,
    '2011':11.1,
    '2012':20.6,
    '2013':19.4,
    '2014':22.8,
    '2015':11.7,
    '2016':12.2
}

In [23]:
data['Temp'] = 0.

In [24]:
for i in data.index.tolist():
    year = str(int(data.loc[i]['Year']))
    data.set_value(i,'Temp',temp[year])

## 2013 Projections using Riegel's formula
http://cs229.stanford.edu/proj2015/247_report.pdf

In [25]:
def fullMarathonTime(halfMarathonTime,data):
    return np.round(halfMarathonTime*((26.219/13.095)**1.06))

In [26]:
halfMarathonId = data[data['Year']==2013].index.tolist()
data['halfMarathonTime'] = np.NaN

#### Recalculate Pace of 2013 rows using Full Marathon Time

In [27]:
for i in halfMarathonId:
    halftime = data.loc[i]['Time']
    data.set_value(i,'halfMarathonTime',halftime)
    fulltime = int(fullMarathonTime(halftime,data))
    data.set_value(i,'Time',fulltime)

    data.set_value(i,'Pace',fulltime/26.219)

## Remove instances of same ID, same year, multiple records
errRunnerIds1 for instances where >2 rows in a given year correspond to the same runner
errRunnerIds2 for instances where 2 rows in a given year correspond to the same runner

In [28]:
runnerIds = set(data['Id'].tolist())

errRunnerIds1 = []
errRunnerIds2 = []
for rid in runnerIds:
    runnerData = data[data['Id']==rid]
    
    if(runnerData['Age'].size > 1):
        yearCount = Counter(runnerData['Year'])
        #print yearCount
        if max(yearCount.values()) > 2:
            errRunnerIds1.append(rid)
        elif max(yearCount.values()) == 2:
            errRunnerIds2.append(rid)

In [30]:
errRunnerIds1

[3327, 6291, 10828, 23068, 23714, 23832, 24624, 26270]

In [31]:
for err in errRunnerIds1:
    if err not in [3327,23714,26270]:
        print data[data['Id']==err].ix[:, data.columns != 'halfMarathonTime'].sort_values(by='Year')

         Id  Age  Rank  Year   Time  Pace  Sex  Temp
10482  6291   40   765  2005  15112   576    1  18.9
10484  6291   60   866  2005  15522   592    1  18.9
10486  6291   35   139  2005  12229   466    1  18.9
10483  6291   40   489  2006  13662   521    1  21.1
10485  6291   40  2707  2013  14852   566    1  19.4
          Id  Age  Rank  Year   Time  Pace  Sex  Temp
16383  10828   25  2105  2007  20192   770    1  22.2
16379  10828   30  1423  2012  15310   584    1  20.6
16382  10828   40  3178  2012  19213   733    1  20.6
16385  10828   35  1785  2012  15971   609    1  20.6
16380  10828   30  2365  2013  14620   557    1  19.4
16381  10828   40   234  2013  11773   449    1  19.4
16384  10828   25  3048  2013  15058   574    1  19.4
          Id  Age  Rank  Year   Time  Pace  Sex  Temp
31128  23068   45  2851  2013  14946   570    1  19.4
31129  23068   40  1802  2013  14161   540    1  19.4
31130  23068   35  1649  2013  14013   534    1  19.4
31127  23068   40  2591  2014  189

# *** MUST CORRECT ***
## *** errRunnerIds1 contains 8 id's, which each have >2 repetitions per id in a given year. The id's [3327,23714,26270] are all for 'private' or 'unknown' runners, therefore these can be disregarded. Still need to write a script to correct the other 5***

In [29]:
len(errRunnerIds2)

101

In [30]:
count = 0
for err in errRunnerIds2:
    if (data[data['Id']==err]['Age'].size== 2):
        count += 2
        print data[data['Id']==err].ix[:, data.columns != 'halfMarathonTime']
        data = data.drop(data[data['Id']==err].index)
        errRunnerIds2.remove(err)
count

    Id  Age  Rank  Year   Time  Pace  Sex  Temp
26  12   20   954  2016  14443   552    0  12.2
27  12   30   588  2016  13575   519    0  12.2
        Id  Age  Rank  Year   Time  Pace  Sex  Temp
2372  1436   25  2777  2016  20907   798    1  12.2
2373  1436   30  2403  2016  18883   721    1  12.2
         Id  Age  Rank  Year   Time  Pace  Sex  Temp
10059  5985   20  1430  2005  18338   699    1  18.9
10060  5985   45  1194  2005  16965   647    1  18.9
         Id  Age  Rank  Year   Time  Pace  Sex  Temp
11886  7319   40  1026  2006  15510   591    0  21.1
11887  7319   40  1030  2006  15524   592    0  21.1
         Id  Age  Rank  Year   Time  Pace  Sex  Temp
13483  8588   20  1786  2006  18341   700    0  21.1
13484  8588   35   680  2006  14319   546    1  21.1
         Id  Age  Rank  Year   Time  Pace  Sex  Temp
14401  9308   15  1079  2007  16024   611    1  22.2
14402  9308   60  1928  2007  19176   731    1  22.2
         Id  Age  Rank  Year   Time  Pace  Sex  Temp
14538  9421

48

In [31]:
len(errRunnerIds2)

77

In [32]:
errRunnerIds2

[92,
 195,
 304,
 347,
 470,
 584,
 653,
 929,
 1007,
 1075,
 1168,
 1203,
 1207,
 1229,
 1302,
 1338,
 1470,
 1531,
 1582,
 1666,
 2293,
 2642,
 3391,
 3503,
 3655,
 4142,
 4196,
 4835,
 4937,
 5123,
 5210,
 5315,
 5585,
 6099,
 6857,
 7109,
 7289,
 7506,
 8174,
 9294,
 9370,
 9721,
 10236,
 10638,
 10888,
 11916,
 12244,
 13104,
 13850,
 13944,
 15610,
 15872,
 15970,
 16803,
 17190,
 17499,
 17817,
 17857,
 18403,
 18709,
 18716,
 18740,
 19479,
 20471,
 20754,
 21700,
 21839,
 22059,
 22729,
 22902,
 23632,
 23917,
 23942,
 24205,
 24253,
 24854,
 27165]

There are 24 instances where a given person ran strictly "twice" in the same year, and did not run in any other years. These 48 rows are deleted, since it cannot be concluded that one row is more reliable than another.

# *** MUST CORRECT errRunnerIds2***
## *** There are still 77 id's that have (exactly) 2 of the same runner in the same year. These 77 id's also have data points in other years. These other data points can be used to decide which one of the two redundant rows is deleted. ***

In [35]:
for err in errRunnerIds2:
    print data[data['Id']==err].ix[:, data.columns != 'halfMarathonTime'].sort_values(by='Year')


     Id  Age  Rank  Year   Time  Pace  Sex  Temp
159  92   40  1612  2004  20299   774    1  20.6
156  92   30  1786  2013  14144   539    1  19.4
157  92   25  1731  2013  14086   537    1  19.4
158  92   30  1567  2016  16147   617    1  12.2
      Id  Age  Rank  Year   Time  Pace  Sex  Temp
328  195   55  2796  2014  19626   749    1  22.8
329  195   25  2796  2014  19626   749    1  22.8
330  195   55  1487  2015  16144   616    1  11.7
327  195   60  2865  2016  21776   832    1  12.2
      Id  Age  Rank  Year   Time  Pace  Sex  Temp
482  304   40  2199  2014  17793   679    1  22.8
483  304   50  2993  2014  20265   773    1  22.8
481  304   50  2707  2016  20341   777    1  12.2
      Id  Age  Rank  Year   Time  Pace  Sex  Temp
554  347   50   878  2007  15441   589    1  22.2
561  347   50  1706  2007  18038   688    1  22.2
556  347   50  1119  2010  15579   594    1  21.7
559  347   30  3637  2013  15432   588    1  19.4
558  347   40  1556  2014  16362   624    1  22.8
560  

## Verify that Age is 'reasonable' given Year and 
Kind of goes hand in hand with Removing duplicate IDs in same year

## Cross-Validate Pace/Time/Rank

In [36]:
years = sorted(set(data['Year']))

for y in years:
    currentYear = data[data['Year']==y].sort_values(by='Rank')

    prevTime = currentYear[:1]['Time'].values[0]
    prevPace = currentYear[:1]['Pace'].values[0]
    prevRank = currentYear[:1]['Rank'].values[0]

    for row in currentYear[1:].iterrows():
        if ((row[1]['Pace'] < prevPace) or (row[1]['Rank'] < prevRank) or (row[1]['Time'] < prevTime)):
            print row[1]['Id'], row[1]['Year']
        prevPace = row[1]['Pace']
        prevRank = row[1]['Rank']
        prevTime = row[1]['Time']

The print statement does not output anything. This validates that the Rank, Time and Pace data is all in order, and that they agree with one another. 

## Verify distance of the marathon using Pace, Time

In [37]:
years = sorted(set(data['Year']))

for y in years:
    currentYear = data[data['Year']==y].sort_values(by='Rank')
    
    for row in currentYear.iterrows():
        if not(26.1365 < (row[1]['Time']/row[1]['Pace']) and (row[1]['Time']/row[1]['Pace']) < 26.2927):
            print row[1]['Id'], row[1]['Year']

Nothing is outputted. All distances based on the recorded Time and Pace are found to be between 26.1365 and 26.2927 miles. 

## 

## Years since Last Race

In [55]:
data['yrsSinceLast'] = 15

In [57]:
def yearsSinceLastRace(runnerId,year,data):
    yrs = 15
    runnerAllYears = data[(data['Id']==runnerId) & (data['Year']<year)].sort_values(by='Year')['Year']
    if( runnerAllYears.size > 0 ):
        yrs = year - max(runnerAllYears)
    return yrs
    

In [58]:
for i in data.index.tolist():
    runnerId = data.loc[i]['Id']
    year = data.loc[i]['Year']
    #data.set_value(i,'totalNumRaces',count)
    data.set_value(i,'yrsSinceLast',int(yearsSinceLastRace(runnerId,year,data)))

## Add Number of Races
Number of races run since 2003

In [41]:
data['raceCount'] = 0

In [42]:
for i in data.index.tolist():
    runnerId = data.loc[i]['Id']
    count = int(data[data['Id']==runnerId]['Id'].size)
    data.set_value(i,'raceCount',count)

## Add Number of Races Ran to date (prev. count)

In [43]:
data['prevRaceCount'] = 0

In [44]:
for i in data.index.tolist():
    runnerId = data.loc[i]['Id']
    d = data[data['Id']==runnerId]

    if d['Id'].size > 1:
        runnerYear = data.loc[i]['Year']
        prevCount = d[d['Year']<runnerYear]['Year'].size
        data.set_value(i,'prevRaceCount',prevCount)

## Normalize data

### Get max values per category per year

In [46]:
ages = set(data['Age'].values)
ages.remove(0)
agemin = min(ages)
agemax = max(ages)

tempbounds = temp.values()
tempmax = max(tempbounds)
tempmin = min(tempbounds)

yearmax = 2017.
yearmin = 2003.

In [60]:
normBounds = {}
for y in sorted(set(data['Year'])):
    d = data[data['Year']==y]
    rankbounds = [max(d['Rank'].values),min(d['Rank'].values)]
    timebounds = [max(d['Time'].values),min(d['Time'].values)]
    pacebounds = [max(d['Pace'].values),min(d['Pace'].values)]
    normBounds[str(y)] = [rankbounds, timebounds, pacebounds]
    
normBounds['Age'] = [agemax, agemin]
normBounds['Temp'] = [tempmax,tempmin]
normBounds['Year'] = [yearmax,yearmin]

data['nTime'] = 0.
data['nPace'] = 0.
data['nRank'] = 0.
data['nAge']  = 0.
data['nTemp'] = 0.
data['nYear'] = 0.
data['nTotalCount'] = 0.
data['nPrevCount'] = 0.
data['nLast'] = 0.
#    year

In [61]:
for i in data.index.tolist():
    year = str(int(data.loc[i]['Year']))
    rank = data.loc[i]['Rank']
    pace = data.loc[i]['Pace']
    time = data.loc[i]['Time']
    age = data.loc[i]['Age']
    temp = data.loc[i]['Temp']
    
    yearsSinceLast = data.loc[i]['yrsSinceLast']
    raceCount = data.loc[i]['raceCount']
    prevCount = data.loc[i]['prevRaceCount']
    
    data.set_value(i,'nTime',((rank - normBounds[year][0][1])/normBounds[year][0][0]))
    data.set_value(i,'nPace',((time - normBounds[year][1][1])/normBounds[year][1][0]))
    data.set_value(i,'nRank',((pace - normBounds[year][2][1])/normBounds[year][2][0]))
    data.set_value(i,'nTemp',((temp - normBounds['Temp'][1])/normBounds['Temp'][0]))
    data.set_value(i,'nAge',((age - normBounds['Age'][1])/normBounds['Age'][0]))
    data.set_value(i,'nYear',((int(year) - normBounds['Year'][1])/normBounds['Year'][0]))

    data.set_value(i,'nTotalCount',(raceCount/15.))
    data.set_value(i,'nPrevCount',(prevCount/14.))
    data.set_value(i,'nLast',(yearsSinceLast/15.))

### Average of previous race times

In [79]:
data['histTime'] = 0.

In [82]:
for i in data.index.tolist():
    runnerId = data.loc[i]['Id']
    year = data.loc[i]['Year']
    prevYears = data[(data['Id']==runnerId) & (data['Year']<year)]
    
    if( prevYears['Age'].size > 0):
        historicalTimeAvg = prevYears['nTime'].sum()/data.loc[i]['prevRaceCount']
        data.set_value(i,'histTime',historicalTimeAvg)

# Data is cleaned. (once everything above here is done. which is ain't yet)
Save data in .pkl file. Split Y1 and Y2 data sets. Randomize data. 

### Average number of marathons per person. 

In [84]:
data.to_csv('out.csv')

In [ ]:
f = open('out.pkl', 'w')
pickle.dump(data,f) 
f.close()